In [2]:
import warnings
warnings.filterwarnings("ignore")

# facebook

import pandas as pd
import pickle
import csv
import numpy as np
from sklearn.preprocessing import LabelEncoder
import pandas as pd
import pickle
import csv
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, mean_squared_error, r2_score
from sklearn.linear_model import LinearRegression

import copy
import numpy as np
import torch
import tqdm
import torch.nn as nn
import torch.nn as nn
import torch.optim as optim

# change this
dataset = 'facebook' # folder and dataset name

cols = ['age', 'UCLA_3item_sum', 'stress', 'depression',
        'Extraversion', 'Agreeableness', 'Conscientiousness', 'Emotional_Stability', 'Openness_to_newexp'] # outcome columns

lr = 0.1

# START

train = pickle.load(open(dataset + '/NMF/run_1/train.pkl', 'rb'))
test = pickle.load(open(dataset + '/NMF/run_1/test.pkl', 'rb'))

temp = pd.concat([train, test]).reset_index(drop=True) # concatenating train and test datasets
df = pd.read_csv('data/' + dataset + '.csv') # loading in actual dataset

df['isFemale'] = df['sex'] == 'Female'

def func(x):
    education = x['education']
    bachelor_plus = ["Master's degree or equivalent", "Professional degree (MD, JD, etc.), doctorate degree or higher"]
    bachelor_minus = ["High school diploma or equivalent", "Vocational/Technical Training"]

    if education in bachelor_plus:
        return 'bachelor+'
    elif education in bachelor_minus:
        return "bachelor-"
    else:
        return 'bachelor'

df['education_bucket'] = df.apply(lambda x : func(x), axis = 1)
label_encoder = LabelEncoder()
df['education_bucket'] = label_encoder.fit_transform(df['education_bucket'])

def func(x):
    income = x['income']
    under = ["$10,000 - $19,999", "$20,000 - $34,999", "$35,000 - $49,999"]
    over = ["$100,000 - $149,999", "More than $150,000"]

    if income in under:
        return '<50k+'
    elif income in over:
        return "100k+"
    else:
        return '$50,000 - $99,999'

df['income_bucket'] = df.apply(lambda x : func(x), axis = 1)
label_encoder = LabelEncoder()
df['income_bucket'] = label_encoder.fit_transform(df['income_bucket'])

label_encoder = LabelEncoder()
df['marital_status'] = label_encoder.fit_transform(df['marital_status'])

merged = pd.merge(temp, df, how='inner', left_on = 'message_id', right_on = 'Unnamed: 0')[['message_id_x', 'user_id', 'message_x'] + cols]
merged.columns = ['message_id', 'user_id', 'message'] + cols

user_ids = {user_id : user_data.index.tolist() for user_id, user_data in merged.groupby('user_id')} # gets all users and their corresponding indices in the df
all_user_ids = list(user_ids.keys())

labels_data = merged[['user_id'] + cols] # df of user_ids and all the labels to consider
user_ids_avg_dict = {} # populated to be user_id : average distribution

all_ys = []
for key in user_ids.keys():
    all_ys.append(labels_data[labels_data['user_id'] == key][cols].iloc[0])

for outcome in cols:
    print(outcome)
    print()

    y = [i[outcome] for i in all_ys]
    
    # REGRESSION

    all_scores = {}

    for model in ['Mallet_LDA']:
        print(model)
        
        train_error_list = []
        test_error_list = []

        r2s_train = []
        r2s_test = []

        for run in range(1, 6):
            print("Run: " + str(run))
            print()

            topics = []
            with open(dataset + '/' + model + '/run_' + str(run) + '/' + 'topics_100.txt', 'r') as f:
                reader = csv.reader(f)
                for row in reader:
                    topic_list = [item.strip() for item in row if item.strip()]
                    topics.append(topic_list)

            X = pickle.load(open(dataset + '/' + model + '/run_' + str(run) + '/' + model + '_avg_topic_distribution.pkl', 'rb'))

            # 80-20 split --> didn't use train-test-split function since its already shuffled
            X_train = X[:round(0.80 * len(X))]
            X_test = X[round(0.80 * len(X)):]

            y_train = y[:round(0.80 * len(X))]
            y_test = y[round(0.80 * len(X)):]

            X_train = np.array(X_train)
            X_test = np.array(X_test)
            y_train = np.array(y_train)
            y_test = np.array(y_test)

            torch.manual_seed(42) # for reproducibility

            X_train_NN = torch.tensor(X_train, dtype = torch.float32)
            y_train_NN = torch.tensor(y_train, dtype = torch.float32).reshape(-1, 1)
            X_test_NN = torch.tensor(X_test, dtype = torch.float32)
            y_test_NN = torch.tensor(y_test, dtype = torch.float32).reshape(-1, 1)

            model_NN = nn.Sequential(
                nn.Linear(X_train_NN.shape[1], 1),
            )

            loss_fn = nn.MSELoss()
            optimizer = optim.Adam(model_NN.parameters(), lr = lr)

            n_epochs = 1000
            batch_size = 32
            batch_start = torch.arange(0, len(X_train), batch_size)

            best_mse = np.inf
            best_weights = None
            history = []

            for epoch in range(n_epochs):
                model_NN.train()
                with tqdm.tqdm(batch_start, unit="batch", mininterval=0, disable=True) as bar:
                    bar.set_description(f"Epoch {epoch}")
                    for start in bar:

                        X_batch = X_train_NN[start : start+batch_size]
                        y_batch = y_train_NN[start : start+batch_size]

                        y_pred = model_NN(X_batch)
                        loss = loss_fn(y_pred, y_batch)

                        optimizer.zero_grad()
                        loss.backward()

                        optimizer.step()

                        bar.set_postfix(mse=float(loss))

                model_NN.eval()
                y_pred = model_NN(X_test_NN)
                mse = loss_fn(y_pred, y_test_NN)
                mse = float(mse)
                history.append(mse)
                if mse < best_mse:
                    best_mse = mse
                    best_weights = copy.deepcopy(model_NN.state_dict())

                if epoch % 50 == 0:
                    print(f'Epoch {epoch+1}, Loss: {loss.item()}')

            model_NN.load_state_dict(best_weights)

            y_pred_train = model_NN(X_train_NN).detach().numpy()
            y_pred_test = model_NN(X_test_NN).detach().numpy()

            train_error = np.sqrt(mean_squared_error(y_pred_train, y_train))
            test_error = np.sqrt(mean_squared_error(y_pred_test, y_test))

            train_error_list.append(train_error)
            test_error_list.append(test_error)

            print(f'Train RMSE: {train_error}')
            print(f'Test RMSE: {test_error}')

            r2_train = r2_score(y_train, y_pred_train)
            r2_test = r2_score(y_test, y_pred_test)

            r2s_train.append(r2_train)
            r2s_test.append(r2_test)
            
            print(f'R2 Train: {r2_train}')
            print(f'R2 Test: {r2_test}')

            all_scores[model] = {
                'Train RMSE': train_error_list,
                'Test RMSE': test_error_list,
                'R2 Train': r2s_train,
                'R2 Test': r2s_test,

                'y_train': y_train,
                'y_train_pred': y_pred_train,
                'y_test': y_test,
                'y_test_pred': y_pred_test
            }
        
        print()

    print()

    for m in all_scores.keys():
        print(f'{m} Average Train RMSE: {np.mean(all_scores[m]["Train RMSE"])}')
        print(f'{m} Average Test RMSE: {np.mean(all_scores[m]["Test RMSE"])}')
        print(f'{m} Average R2 Train: {np.mean(all_scores[m]["R2 Train"])}')
        print(f'{m} Average R2 Test: {np.mean(all_scores[m]["R2 Test"])}')
        print()

    # with open('all_results/' + dataset + '_all_scores_' + outcome + '_NN.pkl', 'wb') as f:
    #     pickle.dump(all_scores, f)

age

Mallet_LDA
Run: 1

Epoch 1, Loss: 1807.4432373046875
Epoch 51, Loss: 846.8967895507812
Epoch 101, Loss: 364.09808349609375
Epoch 151, Loss: 165.1435089111328
Epoch 201, Loss: 101.82080841064453
Epoch 251, Loss: 87.878662109375
Epoch 301, Loss: 86.59911346435547
Epoch 351, Loss: 87.09384155273438
Epoch 401, Loss: 87.38656616210938
Epoch 451, Loss: 87.44734954833984
Train RMSE: 9.732276386240782
Test RMSE: 10.514234920805848
R2 Train: -0.03833259058507599
R2 Test: -0.0012626944879936541
Run: 2



FileNotFoundError: [Errno 2] No such file or directory: 'facebook/Mallet_LDA/run_2/topics_100.txt'